In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer

# import numpy as np

In [2]:
product_info_file_path = '/home/aishwarya/Columbia/Fall 2024/Applied ML/skincare_ml/data/product_info.csv'
data = pd.read_csv(product_info_file_path)
data_info = data.info()
data_head = data.head()

# data_info, data_head
data.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8494 entries, 0 to 8493
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_id          8494 non-null   object 
 1   product_name        8494 non-null   object 
 2   brand_id            8494 non-null   int64  
 3   brand_name          8494 non-null   object 
 4   loves_count         8494 non-null   int64  
 5   rating              8216 non-null   float64
 6   reviews             8216 non-null   float64
 7   size                6863 non-null   object 
 8   variation_type      7050 non-null   object 
 9   variation_value     6896 non-null   object 
 10  variation_desc      1250 non-null   object 
 11  ingredients         7549 non-null   object 
 12  price_usd           8494 non-null   float64
 13  value_price_usd     451 non-null    float64
 14  sale_price_usd      270 non-null    float64
 15  limited_edition     8494 non-null   int64  
 16  new   

Index(['product_id', 'product_name', 'brand_id', 'brand_name', 'loves_count',
       'rating', 'reviews', 'size', 'variation_type', 'variation_value',
       'variation_desc', 'ingredients', 'price_usd', 'value_price_usd',
       'sale_price_usd', 'limited_edition', 'new', 'online_only',
       'out_of_stock', 'sephora_exclusive', 'highlights', 'primary_category',
       'secondary_category', 'tertiary_category', 'child_count',
       'child_max_price', 'child_min_price'],
      dtype='object')

In [4]:
product_info_df = data.copy()
product_info_df.drop(['size', 'variation_type', 'variation_value','variation_desc', 'value_price_usd', 'sale_price_usd', 'limited_edition', 'new', 'online_only', 'out_of_stock', 'sephora_exclusive', 'child_count','child_max_price', 'child_min_price'], axis=1, inplace=True)
product_info_df.rename(columns={'rating': 'overall_rating'}, inplace=True)

# 1. Handle Missing Values
# For numerical columns (e.g., 'rating', 'reviews'), impute with median
numerical_cols = ['overall_rating', 'reviews', 'price_usd']
num_imputer = SimpleImputer(strategy='median')
product_info_df[numerical_cols] = num_imputer.fit_transform(product_info_df[numerical_cols])

In [5]:
# For categorical columns (e.g., 'size', 'variation_type'), impute with most frequent
categorical_cols = ['primary_category', 'secondary_category', 'tertiary_category']
cat_imputer = SimpleImputer(strategy='most_frequent')
product_info_df[categorical_cols] = cat_imputer.fit_transform(product_info_df[categorical_cols])

In [6]:
# # Encoding binary categorical columns
# binary_cols = ['limited_edition', 'new', 'online_only', 'out_of_stock', 'sephora_exclusive']
# product_info_df[binary_cols] = product_info_df[binary_cols].astype(int)

In [7]:
# Encoding specific categorical columns with Label Encoding
label_encoder_cols = ['brand_name', 'primary_category', 'secondary_category', 'tertiary_category']
label_encoders = {col: LabelEncoder() for col in label_encoder_cols}
for col in label_encoder_cols:
    product_info_df[col] = label_encoders[col].fit_transform(product_info_df[col].astype(str))

In [8]:
# # Feature Engineering
# product_info_df['discount_amount'] = product_info_df['price_usd'] - product_info_df['sale_price_usd']
# product_info_df['discount_amount'].fillna(0, inplace=True)

In [9]:
# Standardizing numerical features
scaler = StandardScaler()
product_info_df[numerical_cols] = scaler.fit_transform(product_info_df[numerical_cols])

In [10]:
# Text Processing: TF-IDF Vectorization for 'highlights' and 'ingredients'
product_info_df['highlights'] = product_info_df['highlights'].fillna('')
product_info_df['ingredients'] = product_info_df['ingredients'].fillna('')

# TF-IDF Vectorization with max 50 features to simplify analysis
tfidf_highlights = TfidfVectorizer(max_features=50, stop_words='english')
tfidf_ingredients = TfidfVectorizer(max_features=50, stop_words='english')

# Fit and transform for each column
highlights_tfidf_matrix = tfidf_highlights.fit_transform(product_info_df['highlights'])
ingredients_tfidf_matrix = tfidf_ingredients.fit_transform(product_info_df['ingredients'])

# Convert TF-IDF matrices to DataFrames
highlights_df = pd.DataFrame(highlights_tfidf_matrix.toarray(), columns=tfidf_highlights.get_feature_names_out())
ingredients_df = pd.DataFrame(ingredients_tfidf_matrix.toarray(), columns=tfidf_ingredients.get_feature_names_out())


In [12]:

# Merge TF-IDF features into processed data
product_info_df.drop(['highlights', 'ingredients'], axis=1, inplace=True)
processed_data = pd.concat([product_info_df.reset_index(drop=True), highlights_df, ingredients_df], axis=1)

# Display a sample of the processed data
processed_data_sample = processed_data.head()
processed_data_sample


,product_id,product_name,brand_id,brand_name,loves_count,overall_rating,reviews,price_usd,primary_category,secondary_category,...,potassium,red,root,seed,silica,sodium,stearate,titanium,tocopherol,water
0,P473671,Fragrance Discovery Set,6342,0,6320,-1.103850,-0.393313,-0.310356,1,38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,P473668,La Habana Eau de Parfum,6342,0,3827,-0.086183,-0.391470,2.671043,1,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,P473662,Rainbow Bar Eau de Parfum,6342,0,3253,0.103032,-0.388706,2.671043,1,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,P473660,Kasbah Eau de Parfum,6342,0,3018,0.547941,-0.384098,2.671043,1,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,P473658,Purple Haze Eau de Parfum,6342,0,2691,-1.901619,-0.391470,2.671043,1,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
